# **IEOR 4571 Fall 2020 Homework2 Report**


- Hu, Bo (uni: bh2569)
- Qin, Rui (uni: rq217)
- Yuan, Shuibenyang (uni: sy2938)

In [1]:
%load_ext autoreload
%autoreload 2 

#standard libs
import sys
import pandas as pd
import numpy as np
import pyspark.sql.functions as F
from os import path
import matplotlib.pyplot as plt

#plt env
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 15
fig_size[1] = 5
plt.rcParams["figure.figsize"] = fig_size

#local env
sys.path.append('../')
from src.utils import loading, Spark
data_path = path.join('..', 'data')

# create spark session
spark = Spark()

Spark UI address http://127.0.0.1:4040


# Objective

**Objective function**

The goal is to create lists of movies that users might be interested in. 

**Metrics**

* Root Mean Square Error (RMSE): $RMSE = \sqrt{\frac{\sum_{(u, j) \in E}^{} e_{uj}^2}{\lvert E \rvert}}$, where $e_{ij} = \hat{r}_{uj} - r_{uj}$.

* Accuracy: ratio of ratings such that both predicted value and actual value larger or equal to 3 or both predicted value and actual value less than 3.

* Coverage: ratio of users or movies whose number of both predicted values and actual values larger or equal to 3 larger or equal to k.

**Intended users**

The recommendation system is created for the general audience so that everyone who enjoys movies benefits from the system.

**Business rules**

In order to keep the user entertained rather than only focusing on what they already know, one business rule we come up with is to include at least two different genres when recommending k movies even though the predicted rating might be low. Since our business goal is to benefit all movie lovers, we believe letting them expose to as many genres as possible is fundamental.

**Performance requirements**

For performance, we would like to serve the online queries in real time. For model based algorithms, it’s fine to train it offline and then serve the model online. However, for memory based models, it is hard to meet the performance requirements unless implementing more sophisticated algorithms like approximate nearest neighbor. 

In order to interpret models better, we decide to make the matrix factorization algorithm to only produce non-negative matrices. In that case, we would be able to identify some elements that are important for the algorithm to learn users’ behaviours (higher value in the matrices would produce higher ratings).

# The Sample

**sampling methodology**

We perform sampling w.r.t Conditional Matrix Sampling, in which, we will sample the matrix of $M$ user indices and $N$ movie indices filtering out users who do not have at least $i$ ratings and movies which do not have at least $j$ ratings. If numbers of users and movies are not meet the minimal requirements $M$ and $N$, we will keep sampling process with increased matrix indices number of user and movie until users and movies meet minimal requirements $M$ and $N$.

**generating sample**

to generate sample, run following lines (The sample has been already stored under `[project_directory]/raw/sample.csv`):

```bash
cd [project_directory]
python main.py download sample
```

## Load Sample

In [2]:
# load sample data from '/data/raw/sample.csv'
datas = loading(spark, path.join(data_path, 'raw'))
sample = datas['sample']

## Sample Objectives

In [3]:
print(f'''
            number of data points in the sample: {sample.count()},
            number of unique users in the sample: {sample.select('userId').distinct().count()},
            number of unique movies in the sample: {sample.select('movieId').distinct().count()},
            mean of number of movies a user rated:{sample.groupby('userId').agg(F.count('movieId').alias('cnt')).select(F.mean('cnt')).collect()[0][0]},
            mean of number of users a movie be rated: {sample.groupby('movieId').agg(F.count('userId').alias('cnt')).select(F.mean('cnt')).collect()[0][0]},
            average rating: {sample.select(F.mean('rating')).collect()[0][0]},
            standard deviation of rating: {sample.select(F.stddev('rating')).collect()[0][0]},
            average rating by user: {sample.groupby('userId').agg(F.mean('rating').alias('rating')).select(F.mean('rating')).collect()[0][0]},
            standard deviation of rating by user mean: {sample.groupby('userId').agg(F.mean('rating').alias('rating')).select(F.stddev('rating')).collect()[0][0]},
            average rating by movie: {sample.groupby('movieId').agg(F.mean('rating').alias('rating')).select(F.mean('rating')).collect()[0][0]},
            standard deviation of rating by movie mean: {sample.groupby('movieId').agg(F.mean('rating').alias('rating')).select(F.stddev('rating')).collect()[0][0]}
        ''')


            number of data points in the sample: 444289,
            number of unique users in the sample: 20000,
            number of unique movies in the sample: 1000,
            mean of number of movies a user rated:22.21445,
            mean of number of users a movie be rated: 444.289,
            average rating: 3.562069958968149,
            standard deviation of rating: 1.0467842419621054,
            average rating by user: 3.685826601448731,
            standard deviation of rating by user mean: 0.5225862364055156,
            average rating by movie: 3.290922029378176,
            standard deviation of rating by movie mean: 0.5133362661920545
        


# Methodology

We implemented two types of collaborative filtering techniques:

- Memory Based Collaborative Filtering
    
- Model Based Collaborative Filtering

## Baseline Model

**usage**

The source code of Baseline Model is under the file directory: `[project_directory]/src/baseline/baseline.py`

To use the predictor, `import Baseline() from src.baseline`

```python
###example
#train_schema: DataFrame[userId: string, movieId: string, rating: string]
#test_schema: DataFrame[userId: string, movieId: string, rating: string]
from src.baseline import Baseline
model = Baseline(base='user', usercol='userId', itemcol='movieId', ratingcol='rating')
model.fit(train)
prediction = model.predict(test)
#prediction_schema: DataFrame[userId: string, movieId: string, rating: float, prediction: float]
   
```

**implementation details**

The baseline model makes simple prediction by taking mean of user mean and item mean:

$R$ is defined as $R_{u,i} = \frac{\mu_u + \mu_i}{2}$ where u and i represents user and item accordingly.

## Memory Based Collaborative Filtering

**usage**

The source code of Memory Based Collaborative Filtering is under the file directory: `[project_directory]/src/memory_based/memory_based_cf.py`. 

To use the predictor, `import Memory_based_CF() from src.memory_based`

```python
###example
#spark: the spark session
#train_schema: DataFrame[userId: string, movieId: string, rating: string]
#test_schema: DataFrame[userId: string, movieId: string, rating: string]
from src.memory_based import Memory_based_CF
#user based
model = Memory_based_CF(spark, base='user', usercol='userId', itemcol='movieId', ratingcol='rating')
model.fit(train)
prediction = model.predict(test)
#item based
model = Memory_based_CF(spark, base='item', usercol='userId', itemcol='movieId', ratingcol='rating')
model.fit(train)
prediction = model.predict(test)
#prediction_schema: DataFrame[userId: string, movieId: string, rating: float, prediction: float]
   
```


**implementation details**

The data first transformed into sparse matrix representation, (user by item) if user based and (item by user) if item based.

The the prediction matrix $R$ is trained with following formula:

$R$ is defined as $R_{i, j} = \mu_i + \frac{\sum_{v\in P_i(j)}S(i, v)\cdot (r_{vj} - \mu_v)}{\sum_{v\in P_i(j)}|S(i, v)|}$

where $S$ is the Pearson Similarity Matrix

$S$ is defined as $S_{u,v} = \frac{\sum_{k\in I_u \cap I_v}(r_{uk} - \mu_u)(r_{vk} - \mu_v)}{\sqrt{\sum_{k\in I_u \cap I_v}(r_{uk} - \mu_u)^2}\sqrt{\sum_{k \in I_u \cap I_v}(r_{vk} - \mu_v)^2}}$

The algorithm is implemented with numpy array (for prediction) and scipy csr sparse matrix (for training). 

Every operation uses numpy matrix operations (aka. dot product, norm, etc) which optimizes the computational speed by trading off extra memories (for loop takes $\approx 10$ minutes to train and matrix operations takes $\approx 1$ minutes to train for our experimental sample in user based CF).

**user based collabrative filtering**

When R is (user by item) and S is (user by user), it is User Based Collabrative Filtering

**user based collabrative filtering**

When R is (item by user) and S is (item by item), it is Item Based Collabrative Filtering


## Model Based Collaborative Filtering

**usage**

The source code of Memory Based Collaborative Filtering is under the file directory: `[project_directory]/src/model_based/als.py`. 

To use the predictor, just `import Als() from src.model_based`

```python
###example
#train_schema: DataFrame[userId: string, movieId: string, rating: string]
#test_schema: DataFrame[userId: string, movieId: string, rating: string]

from src.model_based import Als

model = Als(userCol='userId', itemCol='movieId', ratingCol='rating', regParam=.01, seed=0, rank=10)
model.fit(train)
prediction = model.predict(test)

#prediction_schema: DataFrame[userId: string, movieId: string, rating: float, prediction: float]
   
```


**implementation detail**

The data first caseted userId and movieId into integers and then fit into `pyspark.ml.recommendation.ALS`.

Our implementation takes advantages of model based collaborative filtering algorithm implemented in `spark.ml`, in which users and products are described by a small set of latent factors that can be used to predict missing entries `spark.ml` uses the alternating least squares (ALS) algorithm to learn these latent factors.

Since there are many parameters in ALS of `spark.ml`, we will fix `nonnegative = True` since negative rating is not possible in our scenario, and we will only select `regParam`(scale of regulization term) and `rank`(number of hidden factors) to be tuned. (We also tried to tune `maxIter` parameter, but when `maxIter > 20` will blow up memory in our machine with large `rank`, and it takes much longer with nearly the same results, so we will keep `maxIter` with default `=10`.)

# Evaluation

## Cross Validation Setup

**general setup**

we will first setup a fixed splited dataset for all models: baseline, memory-based CF, and model-based CF.

With following splits based on every user:
- train, test : $75\%, 25\%$
- train, test : $50\%, 50\%$
- train, test : $25\%, 25\%$

For example, for (train, test: $75\%, 25\%$), we will retain first (older to newer) $75\%$ of every user's rating w.r.t their rating time as train set, and we will take the rest as test set.

**model based CF**

For model based CF, we will do additional hyper-parameter tuning with $10\%$ validation set from train set to get the best possible parameter.

## Evaluation Metrics

Our metrics are as follow:

* Root Mean Square Error (RMSE): $RMSE = \sqrt{\frac{\sum_{(u, j) \in E}^{} e_{uj}^2}{\lvert E \rvert}}$, where $e_{ij} = \hat{r}_{uj} - r_{uj}$.

* Accuracy: ratio of ratings such that both predicted value and actual value larger or equal to 3 or both predicted value and actual value less than 3.

* Coverage: ratio of users or movies whose number of both predicted values and actual values larger or equal to 3 larger or equal to k.

**choice of k in coverage**

Since mean of number of movies a user rated:22.21445 and mean of number of users a movie be rated: 444.289, We will choose $k = 10$ for user coverage and $k = 100$ for item coverage.

## Paramer Tuning for ALS

we used following parameters as our grid search for paramerter tuning with $10\%$ validation set of train set to get the best possible parameter of `regParam` and `rank`

``` json
parameters = {
    "regParam": [0.01, 0.05, 0.1, 0.15],
    "rank": [10, 50, 100, 150, 200]
}
```

The details of parameter turning can be found in this [notebook](./als_parameter_tuning.ipynb)

In [42]:
tuning_res = pd.read_csv(path.join(data_path, 'processed', 'als_parameter_tuning.csv'))
tuning_res['regParam'] = tuning_res['(regParam, rank)'].str.strip('()').str.split(',').str[0].astype(float)
tuning_res['rank'] = tuning_res['(regParam, rank)'].str.strip('()').str.split(',').str[1].astype(int)
tuning_res = tuning_res.set_index('(regParam, rank)')

In [43]:
tuning_res

,rmse,accuracy,converage_k,split,regParam,rank
"(regParam, rank)",,,,,,
"(0.01, 10)",1.017039,0.664857,0.402209,0.75_0.25,0.01,10
"(0.01, 50)",1.047502,0.622084,0.429391,0.75_0.25,0.01,50
"(0.01, 100)",0.977639,0.643806,0.434425,0.75_0.25,0.01,100
"(0.01, 150)",0.949527,0.648039,0.430899,0.75_0.25,0.01,150
"(0.01, 200)",0.932633,0.656331,0.430247,0.75_0.25,0.01,200
"(0.05, 10)",0.951660,0.655630,0.396881,0.75_0.25,0.05,10
"(0.05, 50)",0.959971,0.623982,0.414841,0.75_0.25,0.05,50
"(0.05, 100)",0.953398,0.622493,0.425069,0.75_0.25,0.05,100
"(0.05, 150)",0.943086,0.628682,0.425328,0.75_0.25,0.05,150


In [44]:
tuning_res[tuning_res.split == '0.75_0.25']

,rmse,accuracy,converage_k,split,regParam,rank
"(regParam, rank)",,,,,,
"(0.01, 10)",1.017039,0.664857,0.402209,0.75_0.25,0.01,10
"(0.01, 50)",1.047502,0.622084,0.429391,0.75_0.25,0.01,50
"(0.01, 100)",0.977639,0.643806,0.434425,0.75_0.25,0.01,100
"(0.01, 150)",0.949527,0.648039,0.430899,0.75_0.25,0.01,150
"(0.01, 200)",0.932633,0.656331,0.430247,0.75_0.25,0.01,200
"(0.05, 10)",0.951660,0.655630,0.396881,0.75_0.25,0.05,10
"(0.05, 50)",0.959971,0.623982,0.414841,0.75_0.25,0.05,50
"(0.05, 100)",0.953398,0.622493,0.425069,0.75_0.25,0.05,100
"(0.05, 150)",0.943086,0.628682,0.425328,0.75_0.25,0.05,150


In [5]:
for split in ['0.75_0.25', '0.5_0.5', '0.25_0.75']:
    subset = tuning_res[tuning_res.split == split].drop('split', axis = 1)
    print(f'''with split = {split}''')
    

,rmse,accuracy,converage_k,split
"(regParam, rank)",,,,
"(0.01, 10)",1.017039,0.664857,0.402209,0.75_0.25
"(0.01, 50)",1.047502,0.622084,0.429391,0.75_0.25
"(0.01, 100)",0.977639,0.643806,0.434425,0.75_0.25
"(0.01, 150)",0.949527,0.648039,0.430899,0.75_0.25
"(0.01, 200)",0.932633,0.656331,0.430247,0.75_0.25
"(0.05, 10)",0.951660,0.655630,0.396881,0.75_0.25
"(0.05, 50)",0.959971,0.623982,0.414841,0.75_0.25
"(0.05, 100)",0.953398,0.622493,0.425069,0.75_0.25
"(0.05, 150)",0.943086,0.628682,0.425328,0.75_0.25


In [6]:
for split in ['0.75_0.25', '0.5_0.5', '0.25_0.75']:
    subset = tuning_res[tuning_res.split == split].drop('split', axis = 1)
    print(f'''with split = {split}''')
    for col in subset.columns:
        if col == 'rmse':
            optimal = subset[col].idxmin()
        else:
            optimal = subset[col].idxmin()
        print(f'''
                    for {col}: 
                    the optimal parameter is {optimal}
                ''')
        subset[col]

with split = 0.75_0.25

                    for rmse: 
                    the optimal parameter is (0.15, 10)
                

                    for accuracy: 
                    the optimal parameter is (0.01, 50)
                

                    for converage_k: 
                    the optimal parameter is (0.05, 10)
                
with split = 0.5_0.5

                    for rmse: 
                    the optimal parameter is (0.15, 10)
                

                    for accuracy: 
                    the optimal parameter is (0.05, 100)
                

                    for converage_k: 
                    the optimal parameter is (0.1, 10)
                
with split = 0.25_0.75

                    for rmse: 
                    the optimal parameter is (0.15, 10)
                

                    for accuracy: 
                    the optimal parameter is (0.05, 50)
                

                    for converage_k: 
                    the opti

## Evaluation Execution

To reduce the tediousness of final report, please see the process of Evaluation Executation in this [notebook](./evaluation_execution.ipynb)